# LINEAR REGRESSION USING PYTORCH BUILT-INS



Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [0]:
import torch
import numpy as np
import torch.nn as nn


In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (mango, pomegranates)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs) 
targets = torch.from_numpy(targets)

## dataset and dataloader

we'll create `TensorDataset`, which allows access to rows from inputs and targets as tuples

In [0]:
from torch.utils.data import TensorDataset

In [0]:
#define dataset
train_ds = TensorDataset(inputs, targets)


In [0]:
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

we'll also create a DataLoader which can split the data into batches of a predefined size while training. It also provides other functionalities like shuffling and random sampling of data.

In [0]:
from torch.utils.data import DataLoader

In [0]:
#define dataloader
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle=True)

In [0]:
train_dl

In [0]:
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 87., 134.,  58.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [102.,  43.,  37.]])
tensor([[119., 133.],
        [119., 133.],
        [ 81., 101.],
        [ 22.,  37.],
        [ 22.,  37.]])


The data loader is used in `for-each`loop. In each iteration, the data loader returns one batch of data,with the given batch size.
if `shuffle=True` it shuffles training data before creating batches.

## nn.linear

we can define model using `nn.linear` class from PyTorch which automatically initialises the weights and biases

In [0]:
#define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2829,  0.3215, -0.1864],
        [-0.3682, -0.4042,  0.5458]], requires_grad=True)
Parameter containing:
tensor([0.4707, 0.1875], requires_grad=True)


`.parameters` method of PyTorch returns a list containing all the weights and bias matrices present in the model.

In [0]:
#parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2829,  0.3215, -0.1864],
         [-0.3682, -0.4042,  0.5458]], requires_grad=True),
 Parameter containing:
 tensor([0.4707, 0.1875], requires_grad=True)]

we can use the model to generate predictions 

In [0]:
#generate predictions
preds = model(inputs)


In [0]:
preds

tensor([[ -6.6603, -30.3033],
        [ -8.9171, -33.9570],
        [  8.1205, -54.3548],
        [-21.4610, -34.5538],
        [ -1.2401, -25.8158],
        [ -6.6603, -30.3033],
        [ -8.9171, -33.9570],
        [  8.1205, -54.3548],
        [-21.4610, -34.5538],
        [ -1.2401, -25.8158],
        [ -6.6603, -30.3033],
        [ -8.9171, -33.9570],
        [  8.1205, -54.3548],
        [-21.4610, -34.5538],
        [ -1.2401, -25.8158]], grad_fn=<AddmmBackward>)

## Loss Function

we can use the built in loss function `mse_loss`

In [0]:
#the nn.functional package contains many useful loss functions 
import torch.nn.functional as F

In [0]:
#define loss function
loss_fn = F.mse_loss

In [0]:
#compute loss for our model
loss = loss_fn(model(inputs), targets)


In [0]:
loss

tensor(12652.8066, grad_fn=<MseLossBackward>)

## Optimizer

we can use the optimizer `optim.SGD`

In [0]:
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

`model.parameters` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step.

## Train the model

We will follow 5 steps - 
1. Generate predictions
2. Calculate the loss
3. Compute gradients with respect to the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. reset the gradients to zero

In [0]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

* we use `opt.step` to perform the update and `opt_zero.grad` to reset the gradients to zero

* `loss.item` returns the actual value stored in the loss tensor 

Lets train the model now for 100 epochs

In [0]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 172.3027
Epoch [20/100], Loss: 138.3775
Epoch [30/100], Loss: 69.2832
Epoch [40/100], Loss: 35.3044
Epoch [50/100], Loss: 75.0550
Epoch [60/100], Loss: 53.5310
Epoch [70/100], Loss: 19.9410
Epoch [80/100], Loss: 19.8258
Epoch [90/100], Loss: 17.1958
Epoch [100/100], Loss: 6.5585


In [0]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.2242,  71.0804],
        [ 79.4001, 101.8216],
        [123.5077, 128.7396],
        [ 26.3170,  41.6629],
        [ 94.0659, 118.4926],
        [ 58.2242,  71.0804],
        [ 79.4001, 101.8216],
        [123.5077, 128.7396],
        [ 26.3170,  41.6629],
        [ 94.0659, 118.4926],
        [ 58.2242,  71.0804],
        [ 79.4001, 101.8216],
        [123.5077, 128.7396],
        [ 26.3170,  41.6629],
        [ 94.0659, 118.4926]], grad_fn=<AddmmBackward>)

In [0]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Outcome

The predictions are quite close to our targets, and now we have a good model to predict yields for our crop!